# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

When solving this task, we expect you'll face (and successfully deal with) some problems or make up the ideas of the model improvement. Some of them are: 

- solving a problem of n-grams frequencies storing for a large corpus;
- taking into account keyboard layout and associated misspellings;
- efficiency improvement to make the solution faster;
- ...

Please don't forget to describe such cases, and what you decided to do with them, in the Justification section.

##### IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity (or just take another dataset). Compare your solution to the Norvig's corrector, and report the accuracies.

#### Useful resources (also included in the archive in moodle):

1. [Possible dataset with N-grams](https://www.ngrams.info/download_coca.asp)
2. [Damerau–Levenshtein distance](https://en.wikipedia.org/wiki/Damerau–Levenshtein_distance#:~:text=Informally%2C%20the%20Damerau–Levenshtein%20distance,one%20word%20into%20the%20other.)

# Solution

There are several approaches to solve this problem. 

## Approach 1: Sequence-to-Sequence Deep Learning Models
These models are effective for capturing the context of the input text. They can complex patterns and relationships in the between words in a sentence and suggest better alternatives for a selected word. Moreover, they also could be trained on subword level and can be used for generating several options for a wrongly spelled word, eventually solving the out-of-vocabulary problem. However, as a downside, these models require a significant amount of data to train and are computationally expensive.

## Approach 2: Word Embedding
Models that produce word embeddings are trained to represent words as vectors in a high-dimensional space. These models are effective for capturing the context of the input text and represending it in a vector space. Once trained, they can be used to produce word embeddings for a given word based only on its context (skip-gram model). Spell correction could be done by producing vector search / clustering based on the word embeddings. However, these models are computationally expensive, requires additional vector space storage and prone to out-of-vocabulary problem (however, they are good at suggesting an alternative word based on the context).

## Approach 3: Norvig Spell Correction
This approach is based on the use of a dictionary of words and their corrections. It is simple and effective, but it is not very accurate. It is also quite slow.

## Approach 4: . Symmetric Delete Spelling Correction Algorithm (SymSpell)
This algorithm is known for its speed and efficiency. It reduces the complexity of edit candidate generation by only considering deletions, making it language-independent and fast. It is significantly faster than traditional algorithms like Norvig's or BK-tree, making it suitable for large-scale applications. However, as a downside, it does not capture complex contextual relationships between words, which can lead to inaccurate suggestions.

## Approach 5: Noisy Channel
These models use probabilistic approaches to correct misspellings by maximizing the probability of the intended word given the misspelled word. They can handle various types of errors, including phonetic and keyboard errors. However, they require accurate models of both word probabilities and error channels.


|Approach|Speed|Contextual Understanding|Complexity|
|---|---|---|---|
|Deep Learning|Slow|High|High|
|Word Embedding|Medium|Medium|Medium|
|Norvig|Medium|Low|Low|
|SymSpell|Fast|Low|Low|
|Noisy Channel|Medium|Medium|Medium|

In this notebook, I will implement Approach 5 - Noisy Channel. Details of the approach can be found here: [Spelling Correction and the Noisy Channel](https://web.stanford.edu/~jurafsky/slp3/B.pdf)

Training data is takes from [1 Billion Word Language Model Benchmark](https://www.statmt.org/lm-benchmark/) 

In [3]:
from collections import Counter, defaultdict
import string
from tqdm import tqdm
import nltk
from pathlib import Path
from typing import Tuple


class NoisyChannelSpellCorrector:
    def __init__(self):
        self.word_counts = Counter()
        self.bigram_counts = defaultdict(int)
        self.vocab = set()
        self.total_words = 0
        self.alphabet = string.ascii_lowercase
        self.similar_words_cache = {}

        self.qwerty_distances = self._init_qwerty_distances()

    def _init_qwerty_distances(self) -> dict[Tuple[str, str], float]:
        keyboard_layout = ["qwertyuiop", "asdfghjkl", "zxcvbnm"]

        char_positions = {}
        for row_idx, row in enumerate(keyboard_layout):
            for col_idx, char in enumerate(row):
                char_positions[char] = (row_idx, col_idx)

        distances = {}
        for c1 in self.alphabet:
            for c2 in self.alphabet:
                if c1 not in char_positions or c2 not in char_positions:
                    distances[(c1, c2)] = 1.0
                    continue

                pos1 = char_positions[c1]
                pos2 = char_positions[c2]
                distance = ((pos1[0] - pos2[0]) ** 2 + (pos1[1] - pos2[1]) ** 2) ** 0.5
                distances[(c1, c2)] = min(distance / 5.0, 1.0)

        return distances

    def _levenshtein_distance(self, s1: str, s2: str) -> float:
        if len(s1) < len(s2):
            return self._levenshtein_distance(s2, s1)

        if len(s2) == 0:
            return len(s1)

        previous_row = range(len(s2) + 1)
        for i, c1 in enumerate(s1):
            current_row = [i + 1]
            for j, c2 in enumerate(s2):
                keyboard_dist = self.qwerty_distances.get((c1, c2), 1.0)
                insertions = previous_row[j + 1] + 1
                deletions = current_row[j] + 1
                substitutions = previous_row[j] + keyboard_dist
                current_row.append(min(insertions, deletions, substitutions))
            previous_row = current_row

        return previous_row[-1]

    def train(self, file_paths: list[str | Path]):
        words = []

        for file_path in file_paths:
            with open(file_path, "rb") as f:
                num_lines = sum(1 for _ in f)

            with open(file_path, "r", encoding="utf-8") as f:
                for line in tqdm(f, desc=f"Processing file: {file_path}", total=num_lines):
                    words.extend(nltk.word_tokenize(line.lower()))

        self.total_words = len(words)
        self.vocab = set(words)
        self.word_counts.update(words)

        padded_words = ["<s>"] + words + ["</s>"]
        for i in tqdm(
            range(len(padded_words) - 1), desc="Building bigrams", unit="bigrams"
        ):
            bigram = tuple(padded_words[i : i + 2])
            self.bigram_counts[bigram] += 1

    def _get_similar_words(self, word: str) -> set[str]:
        if word in self.similar_words_cache:
            return self.similar_words_cache[word]

        splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
        deletes = [L + R[1:] for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
        replaces = [L + c + R[1:] for L, R in splits if R for c in self.alphabet]
        inserts = [L + c + R for L, R in splits for c in self.alphabet]

        candidates = set(deletes + transposes + replaces + inserts)
        self.similar_words_cache[word] = candidates
        return candidates

    def _channel_probability(self, error: str, correction: str) -> float:
        if error == correction:
            return 1.0

        lev_distance = self._levenshtein_distance(error, correction)
        similarity = 1.0 / (1.0 + lev_distance)

        if correction in self._get_similar_words(error):
            similarity *= 2.0

        return similarity

    def _language_model_probability(self, prev_word: str, word: str) -> float:
        bigram = tuple([prev_word, word])
        bigram_count = self.bigram_counts[bigram]
        context_count = self.word_counts[prev_word]

        word_prob = self.word_counts[word] / self.total_words

        if context_count == 0:
            return word_prob

        contextual_prob = (bigram_count + 1) / (context_count + len(self.vocab))
        return contextual_prob * 2 + word_prob

    def _correct_word(self, prev_word: str, word: str) -> str:
        if word in self.vocab:
            return word

        candidates = self._get_similar_words(word)
        candidates = candidates.intersection(self.vocab)

        if not candidates:
            return word

        best_prob = float("-inf")
        best_correction = word

        for candidate in candidates:
            channel_prob = self._channel_probability(word, candidate)
            lang_prob = self._language_model_probability(prev_word, candidate)

            prob = 0.4 * channel_prob + 0.6 * lang_prob

            if prob > best_prob:
                best_prob = prob
                best_correction = candidate

        return best_correction

    def correct(self, text: str) -> str:
        words = nltk.word_tokenize(text.lower())
        corrected_words = []

        prev_word = "<s>"
        for word in words:
            if word in self.vocab:
                corrected = word
            else:
                corrected = self._correct_word(prev_word, word)

            corrected_words.append(corrected)
            prev_word = corrected

        return " ".join(corrected_words)

In [ ]:
!wget "https://storage.yandexcloud.net/dartt0n/share/text-corpus-001-010.txt" # 420MB
# !wget "https://storage.yandexcloud.net/dartt0n/share/text-corpus-001-050.txt" # 2.1GB
# !wget "https://storage.yandexcloud.net/dartt0n/share/text-corpus-001-100.txt" # 4.1GB

In [ ]:
corrector = NoisyChannelSpellCorrector()
corrector.train(["text-corpus-001-010.txt"])

Processing file: text-corpus-001-010.txt:   1%|          | 16204/3062393 [00:00<02:19, 21760.35it/s]

In [7]:
texts = [
    "this jxb is done",
    "dking species",
    "dking sport",
    "doing spxrt",
    "pizza is tastx",
    "vwry beautiful",
    "i lkke nlp",
    "prxjxcts are good",
]

for text in texts:
    corrected = corrector.correct(text)
    print(f"{text} -> {corrected}")

this jxb is done -> this jxb is done
dking species -> eking species
dking sport -> eking sport
doing spxrt -> doing spart
pizza is tastx -> pizza is taste
vwry beautiful -> very beautiful
i lkke nlp -> i like nlp
prxjxcts are good -> prxjxcts are good


## Key Differences
Key differences and benefits compared to Norvig's algorithm:

### Context-Aware Corrections (Bigram Model)
   Norvig:
   - Uses unigram frequencies only
   - P(correction) = count(word) / total_words
   
   This model:
   - Uses bigram frequencies
   - P(correction|previous_word) = count(prev_word, word) / count(prev_word)
   Benefit: Can distinguish between valid words in wrong context
   Example: "this job is dine" → "done" (even though "dine" is valid)

### Weighted Channel Model
   Norvig:
   - Simple edit distance (0, 1, 2)
   - Equal weights for all edits
   
   This model:
   - Keyboard distance-aware edits
   - Levenshtein with weighted substitutions
   - P(error|correction) = 1 / (1 + weighted_distance)
   Benefit: More realistic error modeling
   Example: "vwry" → "very" (higher prob than "vary" due to keyboard)

### Combined Probability Model
   Norvig:
   - P(correction) * P(error|correction)
   
   This model:
   - 0.4 * channel_prob + 0.6 * language_prob
   - Allows tuning for different error types
   Benefit: Better balance between edit distance and context
   Example: Can adjust weights for typing vs context errors

### Vocabulary Handling
   Norvig:
   - Tries to correct every word
   
   This model:
   - Preserves dictionary words unless context suggests otherwise
   - if word in vocab: return word
   Benefit: Faster and preserves correct but rare words
   Example: Proper nouns and valid but uncommon words stay unchanged

## Real-world benefits:

### Better Accuracy for Common Typos
   - Keyboard proximity awareness helps with fat-finger errors
   - Example: "tge" → "the" (higher probability due to 'g' being near 't')

### Context-Sensitive Corrections
   - Can fix grammatical errors that Norvig's can't
   - Example: "their goes" → "there goes"

### Performance Optimization
   - Caching reduces repeated computations
   - Dictionary word preservation reduces unnecessary checks